# Introduction

---



This notebook will download Sea Surface Temperature data for solar event dates and output statistics into CSV files for further analysis

# Imports


In [16]:
#!pip3 install s3fs
#!pip3 install time
#!pip3 install requests
#!pip3 install numpy
#!pip3 install pandas
#!pip3 install xarray
#!pip3 install datetime
#!pip3 install pprint
#!pip3 install cartopy
#!pip3 install zarr
#!pip3 install json
#!pip3 install urllib3!=1.25.0

import os
import re
import time
import requests
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy
import zarr
import datetime as dt
import xarray as xr
import glob
from IPython.display import HTML
from json import dumps
from json import loads
from pprint import pprint

####Harmony Client Install

#!pip3 install harmony
#!pip3 install -U harmony-py

from harmony import BBox
from harmony import Client
from harmony import Collection
from harmony import Request 
from harmony import LinkType
from harmony.config import Environment


# Reading From Data Text File

In [100]:
with open('Text Files/SolarEventsDateTime.txt') as f:
    contents = f.read()
    print(contents)

Solar Events in Greenwich Mean Time
  Tropical Year = 365.2425 (days)

         Vernal      Summer      Autumnal     Winter    
   Year     Equinox    Solstice     Equinox     Solstice   Perihelion    Aphelion 
   ----  ----------  ----------  ----------   ----------   ----------   ----------
   2000  3/20  7:30  6/21  1:44  9/22 17:16  12/21 13:27   1/03 23:21   7/04 14:15
   2001  3/20 13:19  6/21  7:32  9/22 23:05  12/21 19:16   1/03  5:35   7/04 20:29
   2002  3/20 19:08  6/21 13:20  9/23  4:54  12/22  1:06   1/03 11:49   7/05  2:43
   2003  3/21  0:58  6/21 19:08  9/23 10:42  12/22  6:56   1/03 18:03   7/05  8:57
   2004  3/20  6:47  6/21  0:56  9/22 16:31  12/21 12:46   1/04  0:17   7/04 15:11
   2005  3/20 12:36  6/21  6:44  9/22 22:20  12/21 18:35   1/03  6:31   7/04 21:25
   2006  3/20 18:25  6/21 12:32  9/23  4:08  12/22  0:25   1/03 12:45   7/05  3:39
   2007  3/21  0:14  6/21 18:21  9/23  9:57  12/22  6:15   1/03 18:59   7/05  9:53
   2008  3/20  6:04  6/21  0:09  9/22 15:4

# Setting up Event Selection

---



---

In [31]:
# Create download directories
if not os.path.isdir('downloads/vernal_equinox/'):
    os.makedirs('downloads/vernal_equinox/')
if not os.path.isdir('downloads/summer_solstice/'):
    os.makedirs('downloads/summer_solstice/')
if not os.path.isdir('downloads/autumnal_equinox/'):
    os.makedirs('downloads/autumnal_equinox/')
if not os.path.isdir('downloads/winter_solstice/'):
    os.makedirs('downloads/winter_solstice/')
if not os.path.isdir('downloads/perihelion/'):
    os.makedirs('downloads/perihelion/')
if not os.path.isdir('downloads/aphelion/'):
    os.makedirs('downloads/aphelion/')

In [32]:
vernal_equinox = []
summer_solstice = []
autumnal_equinox = []
winter_solstice = []
perihelion = []
aphelion = []

with open('Text Files/SolarEventsDateTime.txt') as f:
    contents = f.read()

    for line in contents.splitlines():
        if len(line) > 0:
            values = line.split()
            if values[0].isdigit():
                year = values[0]
                vernal_equinox_date = values[1] 
                summer_solstice_date = values[3]
                autumnal_equinox_date = values[5]
                winter_solstice_date = values[7]
                perihelion_date = values[9]
                aphelion_date = values[11]

                vernal_equinox.append(datetime.strptime(year + '/' + vernal_equinox_date , '%Y/%m/%d').date())
                summer_solstice.append(datetime.strptime(year + '/' + summer_solstice_date , '%Y/%m/%d').date())
                autumnal_equinox.append(datetime.strptime(year + '/' + autumnal_equinox_date , '%Y/%m/%d').date())
                winter_solstice.append(datetime.strptime(year + '/' + winter_solstice_date , '%Y/%m/%d').date())
                perihelion.append(datetime.strptime(year + '/' + perihelion_date , '%Y/%m/%d').date())
                aphelion.append(datetime.strptime(year + '/' + aphelion_date , '%Y/%m/%d').date())

print(vernal_equinox)
print(summer_solstice)
print(autumnal_equinox)
print(winter_solstice)
print(perihelion)
print(aphelion)    

[datetime.date(2000, 3, 20), datetime.date(2001, 3, 20), datetime.date(2002, 3, 20), datetime.date(2003, 3, 21), datetime.date(2004, 3, 20), datetime.date(2005, 3, 20), datetime.date(2006, 3, 20), datetime.date(2007, 3, 21), datetime.date(2008, 3, 20), datetime.date(2009, 3, 20), datetime.date(2010, 3, 20), datetime.date(2011, 3, 20), datetime.date(2012, 3, 20), datetime.date(2013, 3, 20), datetime.date(2014, 3, 20), datetime.date(2015, 3, 20), datetime.date(2016, 3, 20), datetime.date(2017, 3, 20), datetime.date(2018, 3, 20), datetime.date(2019, 3, 20), datetime.date(2020, 3, 20)]
[datetime.date(2000, 6, 21), datetime.date(2001, 6, 21), datetime.date(2002, 6, 21), datetime.date(2003, 6, 21), datetime.date(2004, 6, 21), datetime.date(2005, 6, 21), datetime.date(2006, 6, 21), datetime.date(2007, 6, 21), datetime.date(2008, 6, 21), datetime.date(2009, 6, 21), datetime.date(2010, 6, 21), datetime.date(2011, 6, 21), datetime.date(2012, 6, 20), datetime.date(2013, 6, 21), datetime.date(2014

In [102]:
from harmony import Client

# Connect to Earthdata using Harmony client and download files
harmony_client = Client(auth=('CHANGEME', 'CHANGEME'))
base_url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/'

dates_dict = {'vernal_equinox': vernal_equinox,
              'summer_solstice': summer_solstice,
              'autumnal_equinox': autumnal_equinox,
              'winter_solstice': winter_solstice,
              'perihelion': perihelion,
              'aphelion': aphelion}

for key in dates_dict:
    print(f'Downloading {key} datasets')
    for day in dates_dict[key]:
        print(f'Retrieving {day}')
        filename = day.strftime('%Y%m%d') + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
        if os.path.exists(f'downloads/{key}/{filename}'):
            print(f'File already downloaded: downloads/{key}/{filename}')
            continue
        url = base_url + filename
        print(url)
        r = requests.get(url, allow_redirects=True)
        print(r.status_code)
        if r.status_code == 200:
            open(f'downloads/{key}/{filename}', 'wb').write(r.content)


Retrieving 2000-03-20
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20000320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
404
Retrieving 2001-03-20
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20010320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
404
Retrieving 2002-03-20
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20020320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
404
Retrieving 2003-03-21
File already downloaded: downloads/vernal_equinox/20030321090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2004-03-20
File already downloaded: downloads/vernal_equinox/20040320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2005-03-20
File already downloaded: downloads/vernal_equinox/20050320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2006-03-20
File already downloaded: downloads/vernal_

404
Retrieving 2001-12-21
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20011221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
404
Retrieving 2002-12-22
File already downloaded: downloads/winter_solstice/20021222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2003-12-22
File already downloaded: downloads/winter_solstice/20031222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2004-12-21
File already downloaded: downloads/winter_solstice/20041221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2005-12-21
File already downloaded: downloads/winter_solstice/20051221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2006-12-22
File already downloaded: downloads/winter_solstice/20061222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 2007-12-22
File already downloaded: downloads/winter_solstice/20071222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
Retrieving 20

200
Retrieving 2008-07-04
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20080704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
200
Retrieving 2009-07-04
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20090704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
200
Retrieving 2010-07-05
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20100705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
200
Retrieving 2011-07-05
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20110705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
200
Retrieving 2012-07-04
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20120704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
200
Retrieving 2013-07-04
https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/MUR-JPL-

# ⭐ Generating CSV Outputs ⭐ 

---



---

# Vernal Equinox

In [111]:
# Loop through all downloaded vernal_equinox files

vernal_equinox_events = []
vernal_equinox_times = []
vernal_equinox_regions = []
vernal_equinox_mean = []
vernal_equinox_max = []
vernal_equinox_min = []
vernal_equinox_mode = []
vernal_equinox_std = []

vernal_equinox_files = sorted(glob.glob('downloads/vernal_equinox/*.nc'))
for vernal_equinox_file in vernal_equinox_files:
    print(vernal_equinox_file)
    data = xr.open_dataset(vernal_equinox_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    vernal_equinox_events.append('Vernal Equinox')
    vernal_equinox_times.append(ds_sst.coords['time'].values[0])
    vernal_equinox_regions.append('California Coast')
    vernal_equinox_mean.append(sst.mean())
    vernal_equinox_max.append(sst.max())
    vernal_equinox_min.append(sst.min())
    vernal_equinox_mode.append(sst.mode())
    vernal_equinox_std.append(sst.std())

# Build aggregate dataframe
vernal_equinox_data = pd.DataFrame({
    "Event": vernal_equinox_events
    ,"Time": vernal_equinox_times
    ,"Region": vernal_equinox_regions
    ,"Standard Deviation": vernal_equinox_std
    ,"Min": vernal_equinox_min
    ,"Max": vernal_equinox_max
    ,"Mean": vernal_equinox_mean})

print(vernal_equinox_data)
# Export to CSV
vernal_equinox_data.to_csv('CSV Files/vernal_equinox_data.csv', index=False)

downloads/vernal_equinox/20030321090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20040320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20050320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20060320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20070321090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20080320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20090320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20100320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20110320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20120320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20130320090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/vernal_equinox/20140320090000-JPL-L4_GHRSST-

# Summer Solstice

In [110]:
# Loop through all downloaded summer_solstice files

summer_solstice_events = []
summer_solstice_times = []
summer_solstice_regions = []
summer_solstice_mean = []
summer_solstice_max = []
summer_solstice_min = []
summer_solstice_mode = []
summer_solstice_std = []

summer_solstice_files = sorted(glob.glob('downloads/summer_solstice/*.nc'))
for summer_solstice_file in summer_solstice_files:
    print(summer_solstice_file)
    data = xr.open_dataset(summer_solstice_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    summer_solstice_events.append('Summer Solstice')
    summer_solstice_times.append(ds_sst.coords['time'].values[0])
    summer_solstice_regions.append('California Coast')
    summer_solstice_mean.append(sst.mean())
    summer_solstice_max.append(sst.max())
    summer_solstice_min.append(sst.min())
    summer_solstice_mode.append(sst.mode())
    summer_solstice_std.append(sst.std())

# Build aggregate dataframe
summer_solstice_data = pd.DataFrame({
    "Event": summer_solstice_events
    ,"Time": summer_solstice_times
    ,"Region": summer_solstice_regions
    ,"Standard Deviation": summer_solstice_std
    ,"Min": summer_solstice_min
    ,"Max": summer_solstice_max
    ,"Mean": summer_solstice_mean})

print(summer_solstice_data)
# Export to CSV
summer_solstice_data.to_csv('CSV Files/summer_solstice_data.csv', index=False)

downloads/summer_solstice/20020621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20030621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20040621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20050621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20060621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20070621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20080621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20090621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20100621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20110621090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20120620090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/summer_solstice/20130621090000-JP

# Autumnal Equinox

In [109]:
# Loop through all downloaded autumnal_equinox files

autumnal_equinox_events = []
autumnal_equinox_times = []
autumnal_equinox_regions = []
autumnal_equinox_mean = []
autumnal_equinox_max = []
autumnal_equinox_min = []
autumnal_equinox_mode = []
autumnal_equinox_std = []

autumnal_equinox_files = sorted(glob.glob('downloads/autumnal_equinox/*.nc'))
for autumnal_equinox_file in autumnal_equinox_files:
    print(autumnal_equinox_file)
    data = xr.open_dataset(autumnal_equinox_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    autumnal_equinox_events.append('Autumnal Equinox')
    autumnal_equinox_times.append(ds_sst.coords['time'].values[0])
    autumnal_equinox_regions.append('California Coast')
    autumnal_equinox_mean.append(sst.mean())
    autumnal_equinox_max.append(sst.max())
    autumnal_equinox_min.append(sst.min())
    autumnal_equinox_mode.append(sst.mode())
    autumnal_equinox_std.append(sst.std())

# Build aggregate dataframe
autumnal_equinox_data = pd.DataFrame({
    "Event": autumnal_equinox_events
    ,"Time": autumnal_equinox_times
    ,"Region": autumnal_equinox_regions
    ,"Standard Deviation": autumnal_equinox_std
    ,"Min": autumnal_equinox_min
    ,"Max": autumnal_equinox_max
    ,"Mean": autumnal_equinox_mean})

print(autumnal_equinox_data)
# Export to CSV
autumnal_equinox_data.to_csv('CSV Files/autumnal_equinox_data.csv', index=False)

downloads/autumnal_equinox/20020923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20030923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20040922090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20050922090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20060923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20070923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20080922090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20090922090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20100923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20110923090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20120922090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/autumnal_equinox/20130

# Winter Solstice

In [108]:
# Loop through all downloaded winter_solstice files

winter_solstice_events = []
winter_solstice_times = []
winter_solstice_regions = []
winter_solstice_mean = []
winter_solstice_max = []
winter_solstice_min = []
winter_solstice_mode = []
winter_solstice_std = []

winter_solstice_files = sorted(glob.glob('downloads/winter_solstice/*.nc'))
for winter_solstice_file in winter_solstice_files:
    print(winter_solstice_file)
    data = xr.open_dataset(winter_solstice_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    winter_solstice_events.append('Winter Solstice')
    winter_solstice_times.append(ds_sst.coords['time'].values[0])
    winter_solstice_regions.append('California Coast')
    winter_solstice_mean.append(sst.mean())
    winter_solstice_max.append(sst.max())
    winter_solstice_min.append(sst.min())
    winter_solstice_mode.append(sst.mode())
    winter_solstice_std.append(sst.std())

# Build aggregate dataframe
winter_solstice_data = pd.DataFrame({
    "Event": winter_solstice_events
    ,"Time": winter_solstice_times
    ,"Region": winter_solstice_regions
    ,"Standard Deviation": winter_solstice_std
    ,"Min": winter_solstice_min
    ,"Max": winter_solstice_max
    ,"Mean": winter_solstice_mean})

print(winter_solstice_data)
# Export to CSV
winter_solstice_data.to_csv('CSV Files/winter_solstice_data.csv', index=False)

downloads/winter_solstice/20021222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20031222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20041221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20051221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20061222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20071222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20081221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20091221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20101221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20111222090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20121221090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/winter_solstice/20131221090000-JP

# Perihelion

In [107]:
# Loop through all downloaded perihelion files

perihelion_events = []
perihelion_times = []
perihelion_regions = []
perihelion_mean = []
perihelion_max = []
perihelion_min = []
perihelion_mode = []
perihelion_std = []

perihelion_files = sorted(glob.glob('downloads/perihelion/*.nc'))
for perihelion_file in perihelion_files:
    print(perihelion_file)
    data = xr.open_dataset(perihelion_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    perihelion_events.append('Perihelion')
    perihelion_times.append(ds_sst.coords['time'].values[0])
    perihelion_regions.append('California Coast')
    perihelion_mean.append(sst.mean())
    perihelion_max.append(sst.max())
    perihelion_min.append(sst.min())
    perihelion_mode.append(sst.mode())
    perihelion_std.append(sst.std())

# Build aggregate dataframe
perihelion_data = pd.DataFrame({
    "Event": perihelion_events
    ,"Time": perihelion_times
    ,"Region": perihelion_regions
    ,"Standard Deviation": perihelion_std
    ,"Min": perihelion_min
    ,"Max": perihelion_max
    ,"Mean": perihelion_mean})

print(perihelion_data)
# Export to CSV
perihelion_data.to_csv('CSV Files/perihelion_data.csv', index=False)

downloads/perihelion/20030103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20040104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20050103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20060103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20070103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20080104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20090103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20100103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20110103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20120104090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20130103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihelion/20140103090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/perihe

# Aphelion

In [106]:
# Loop through all downloaded aphelion files

aphelion_events = []
aphelion_times = []
aphelion_regions = []
aphelion_mean = []
aphelion_max = []
aphelion_min = []
aphelion_mode = []
aphelion_std = []

aphelion_files = sorted(glob.glob('downloads/aphelion/*.nc'))
for aphelion_file in aphelion_files:
    print(aphelion_file)
    data = xr.open_dataset(aphelion_file)
    
    # Subset the data to study area
    lat_bnds, lon_bnds = [26, 42], [-130, -110]
    data_subset = data.sel(lat=slice(*lat_bnds), lon=slice(*lon_bnds))

    # Assigning Variable to Gridded Dataset of Only SST Values
    ds_sst = data_subset.analysed_sst
    df_sst = ds_sst.to_dataframe()
    sst = df_sst['analysed_sst']
    
    # Append values to arrays
    aphelion_events.append('Aphelion')
    aphelion_times.append(ds_sst.coords['time'].values[0])
    aphelion_regions.append('California Coast')
    aphelion_mean.append(sst.mean())
    aphelion_max.append(sst.max())
    aphelion_min.append(sst.min())
    aphelion_mode.append(sst.mode())
    aphelion_std.append(sst.std())

# Build aggregate dataframe
aphelion_data = pd.DataFrame({
    "Event": aphelion_events
    ,"Time": aphelion_times
    ,"Region": aphelion_regions
    ,"Standard Deviation": aphelion_std
    ,"Min": aphelion_min
    ,"Max": aphelion_max
    ,"Mean": aphelion_mean})

print(aphelion_data)
# Export to CSV
aphelion_data.to_csv('CSV Files/aphelion_data.csv', index=False)

downloads/aphelion/20020705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20030705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20040704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20050704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20060705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20070705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20080704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20090704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20100705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20110705090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20120704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20130704090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc
downloads/aphelion/20140705090000-JPL-L4